In [1]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [2]:
filepath = Path("../sample_files/ellipsoid_2.stl")
lv = LV.from_file(filepath)
# lv.tetrahedralize(backend=TETRA_BACKEND.TETGEN, quality=True, order=1, mindihedral=10, minratio=1.4, nobisect=True)
lv.plot(show_edges=True)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [3]:
lv.identify_regions(LV_GEO_TYPES.IDEAL)

In [4]:
lv.transform_region_to_facet_data(LV_MESH_DATA.SURFS)

In [5]:
lv.plot("surface", 
        LV_MESH_DATA.SURFS,
        preference="cell",
        show_edges=True,
        vnodes=[
          (LV_VIRTUAL_NODES.APEX, "green"),
          (LV_VIRTUAL_NODES.BASE, "red"),
          ],
        cmap="Set1",
        
        categorical=True,
        split_sharp_edges=True,
        smooth_shading=False,
        # ambient=0.0,
        # diffuse=0.0,
        # specular=0.0,
        )

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', posit…

In [ ]:
lv.plot("mesh", 
        LV_MESH_DATA.SURFS,
        preference="cell",
        show_edges=True,
        vnodes=[
          (LV_VIRTUAL_NODES.APEX, "green"),
          (LV_VIRTUAL_NODES.BASE, "red"),
          ],
        cmap="Set1",
        
        categorical=True,
        split_sharp_edges=True,
        smooth_shading=False,
        # ambient=0.0,
        # diffuse=0.0,
        # specular=0.0,
        )

In [ ]:
np.unique(lv.mesh.cell_data[LV_MESH_DATA.SURFS.value])

In [ ]:
e1 = 45
b1 = 1
b2 = 89

endo_epi = np.copy(lv.get(GEO_DATA.SURF_POINT_DATA, LV_MESH_DATA.EPI_ENDO_GUESS))
lvsurf = lv.get_surface_mesh()
# Get edges
edges = lv.mesh.extract_feature_edges(e1)
edge_pts = edges.points
est_base = centroid(edge_pts)
est_radius = radius(edge_pts)
# select pts close to est_base based on dist threshold on axis
pts = lvsurf.points
lvnormal = lv.get_normal()
axis = np.where(lvnormal==np.max(lvnormal))[0]
d_base = np.abs(est_base[axis] - pts[:, axis])
ioi = np.where(d_base <= 0.1)[0] 
# print(ioi)
# filter selected pts based on surface angle
surfnormals = lvsurf.compute_normals(inplace=True)
surf_normals = lvsurf.get_array("Normals", "points")
# base_angles = lv.compute_angles_wrt_normal(surf_normals[ioi], check_orientation=False, degrees=False)
len(ioi)
zvec = np.repeat(np.expand_dims(lv._Z, 1), len(ioi), axis=1).T
base_angles = angle_between(surf_normals[ioi], zvec, check_orientation=False)
ioi = ioi[np.where(base_angles <= np.radians(90))[0]]
print(len(ioi))
# filter by endo
ioi = ioi[np.where(endo_epi[ioi] != LV_SURFS.ENDO)]
print(len(ioi))

# identify final surfaces
endo_epi_base = np.copy(endo_epi)*100
endo_epi_base[ioi] = LV_SURFS.BASE
# map to 'global' mesh ids
id_map = lv.get_surface_id_map_from_mesh()
endo_epi_base_mesh = np.zeros(lv.mesh.n_points)
endo_epi_base_mesh[id_map] = endo_epi_base
# add data to mesh
# lv.surface_mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base
# lv.mesh.point_data[LV_MESH_DATA.SURFS.value] = endo_epi_base_mesh

# compute facet data
# cellMap = lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS, method=lambda x: int(np.max(x)), surface=True)
# lv.set_facet_data("FACETS", cellMap)
# lv.transform_surface_point_data_to_facet_data(LV_MESH_DATA.SURFS)

# lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS, method=lambda x: int(np.max(x)))
#plot
# lv.plot("surface", 
#         LV_MESH_DATA.SURFS,
#         show_edges=True,
#         vnodes=[
#           (LV_VIRTUAL_NODES.APEX, "green"),
#           (LV_VIRTUAL_NODES.BASE, "red"),
#           ],
#         cmap="Set1",
#         preference="cell",
#         categorical=True,
#         split_sharp_edges=True,
#         smooth_shading=False,
#         ambient=0.0,
#         diffuse=0.0,
#         specular=0.0,
#         )

In [ ]:
lv.surface_mesh.plot(scalars="FACETS")

In [ ]:
np.unique(cellMap)

In [ ]:
lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS, method=lambda x: int(np.max(x)),)

In [ ]:
lv.transform_point_data_to_cell_data(LV_MESH_DATA.SURFS, method=lambda x: int(np.max(x)), surface=True)

In [ ]:
lvsurf.cells_dict

In [ ]:
lv.compute_fibers(surfRegionsIds=LV_MESH_DATA.SURFS,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={
                      "lv": LV_SURFS.ENDO.value,
                      "epi": LV_SURFS.EPI.value,
                      "base": LV_SURFS.BASE.value
                    },
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

In [ ]:
lv.regress_fibers(lv, hidden_layer_sizes=(100,50), verbose=True, max_iter=250, validation_fraction=0.25, tol=1e-5)

In [ ]:
lv.transform_point_data_to_cell_data(LV_FIBERS.F0, "mean", axis=0)
lv.transform_point_data_to_cell_data(LV_FIBERS.S0, "mean", axis=0)
lv.transform_point_data_to_cell_data(LV_FIBERS.N0, "mean", axis=0)

In [ ]:
lv.mesh.save("./test3.vtk")

In [ ]:
lv.plot_streamlines(LV_FIBERS.F0, decimate_boundary=0.1)

In [ ]:
lv.mesh.save("./test2.vtk")